In [1]:
import pandas as pd
import numpy as np

In [2]:
df_train = pd.read_csv('../data/new_data/train_set.csv')
df_test = pd.read_csv('../data/new_data/test_set.csv')

## 特征工程

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

word_vec = TfidfVectorizer(analyzer='word',
            ngram_range=(1,2),
            min_df=3,  # 低频词
            max_df=0.9,  # 高频词
            use_idf=True,
            smooth_idf=True, 
            sublinear_tf=True)

In [5]:
train_doc = word_vec.fit_transform(df_train['word_seg'])
test_doc = word_vec.transform(df_test['word_seg'])

In [6]:
print(type(train_doc))

<class 'scipy.sparse.csr.csr_matrix'>


## 模型训练

In [7]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
df_train['label'] = lb.fit_transform(df_train['class'])
lb.fit_transform(df_train['class'])

array([13,  2, 11, ..., 11,  3, 10], dtype=int64)

In [8]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=4, solver='liblinear', dual=True) 
clf.fit(train_doc, df_train['label'])

LogisticRegression(C=4, class_weight=None, dual=True, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

## 预测提交

In [9]:
test_prob = clf.predict_proba(test_doc)

In [10]:
test_pred = np.argmax(test_prob, axis=1)
df_test['class'] = lb.inverse_transform(test_pred)
df_test[["id","class"]].to_csv("submission_traditional_baseline.csv", index=False, header=True, encoding='utf-8')